In [ ]:
! pip install transformers

In [2]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
# tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [ ]:
# translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="kaz_Cyrl", tgt_lang="eng_Latn")

In [ ]:
# translator(["Сәлем достар!", "Кетші"])

[{'translation_text': 'Greetings to you, friends!'},
 {'translation_text': 'Going to the'}]

In [ ]:
# from datasets import load_dataset

# flores_dataset = load_dataset("facebook/flores", "eng_Latn-kaz_Cyrl")

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

In [ ]:
# test_data = flores_dataset["devtest"]

In [ ]:
# reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in test_data['sentence_eng_Latn']]


[['▁"', 'We', '▁now', '▁have', '▁4-', 'mon', 'th', '-', 'old', '▁mi', 'ce', '▁that', '▁are', '▁non', '-', 'dia', 'bet', 'ic', '▁that', '▁used', '▁to', '▁be', '▁diabet', 'ic', ',', '"', '▁he', '▁added', '.']]


In [ ]:
# from nltk.translate.bleu_score import sentence_bleu

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
import torch

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

# Initialize translator pipeline
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="kaz_Cyrl", tgt_lang="eng_Latn", batch_size=64, device=device)

# Load dataset
flores_dataset = load_dataset("facebook/flores", "eng_Latn-kaz_Cyrl")
test_data = flores_dataset["devtest"]

# Tokenize reference sentences
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in test_data['sentence_eng_Latn']]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for facebook/flores contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/flores
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data['sentence_kaz_Cyrl'][i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  0  is  1.7354006834793989e-78
bleu score for sentence  1  is  0.5836699724322835
bleu score for sentence  2  is  0.18788079165231159
bleu score for sentence  3  is  0.4527768866102048


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  4  is  4.602347314045479e-155
bleu score for sentence  5  is  0.1935961297781464


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  6  is  1.5656618337072542e-231
bleu score for sentence  7  is  2.427154408681538e-78
bleu score for sentence  8  is  0.7138957847176474
bleu score for sentence  9  is  3.6063918949764824e-78


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bleu score for sentence  10  is  0.37625699624203157
bleu score for sentence  11  is  0.47618920227599343
bleu score for sentence  12  is  0.4695423719539182
bleu score for sentence  13  is  0.27616109500255037
bleu score for sentence  14  is  0.22748398467289185
bleu score for sentence  15  is  0.3242352306426039
bleu score for sentence  16  is  0.27832397701877376
bleu score for sentence  17  is  0.42740577972428406
bleu score for sentence  18  is  0.6944251213831509
bleu score for sentence  19  is  0.1567189408571392
bleu score for sentence  20  is  0.764786017043723
bleu score for sentence  21  is  0.48593738187963054
bleu score for sentence  22  is  4.0798362905540435e-78
bleu score for sentence  23  is  0.5042700033119852
bleu score for sentence  24  is  4.358083314364454e-78
bleu score for sentence  25  is  0.5198973162544194
bleu score for sentence  26  is  0.24452946438904896
bleu score for sentence  27  is  0.5052234990361756
bleu score for sentence  28  is  0.177263691333161

In [ ]:
test_data['sentence_kaz_Cyrl'][1010]

'Жұмыс орнындағы үйлесімділік өте маңызды, себебі жеке жетістіктерді марапаттауға емес, ал топтық жұмысқа назар аударылады.'

In [ ]:
translator(test_data['sentence_kaz_Cyrl'][1010])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'translation_text': 'Workplace consistency is important because it focuses on teamwork rather than individual achievement awards.'}]

In [ ]:
reference_sentences[1010]

[['▁Work',
  'place',
  '▁harm',
  'ony',
  '▁is',
  '▁crucial',
  ',',
  '▁emph',
  'asi',
  'zing',
  '▁group',
  '▁effort',
  '▁rather',
  '▁than',
  '▁pra',
  'ising',
  '▁individual',
  '▁accompl',
  'ish',
  'ments',
  '.']]

In [ ]:
reference_sentences = [[sentence.split()] for sentence in test_data['sentence_eng_Latn']]


[[['"We', 'now', 'have', '4-month-old', 'mice', 'that', 'are', 'non-diabetic', 'that', 'used', 'to', 'be', 'diabetic,"', 'he', 'added.']], [['Dr.', 'Ehud', 'Ur,', 'professor', 'of', 'medicine', 'at', 'Dalhousie', 'University', 'in', 'Halifax,', 'Nova', 'Scotia', 'and', 'chair', 'of', 'the', 'clinical', 'and', 'scientific', 'division', 'of', 'the', 'Canadian', 'Diabetes', 'Association', 'cautioned', 'that', 'the', 'research', 'is', 'still', 'in', 'its', 'early', 'days.']], [['Like', 'some', 'other', 'experts,', 'he', 'is', 'skeptical', 'about', 'whether', 'diabetes', 'can', 'be', 'cured,', 'noting', 'that', 'these', 'findings', 'have', 'no', 'relevance', 'to', 'people', 'who', 'already', 'have', 'Type', '1', 'diabetes.']], [['On', 'Monday,', 'Sara', 'Danius,', 'permanent', 'secretary', 'of', 'the', 'Nobel', 'Committee', 'for', 'Literature', 'at', 'the', 'Swedish', 'Academy,', 'publicly', 'announced', 'during', 'a', 'radio', 'program', 'on', 'Sveriges', 'Radio', 'in', 'Sweden', 'the', 'c

In [ ]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data['sentence_kaz_Cyrl'][i])[0]['translation_text']
  translated_tokens = translated_sentence.split()
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bleu score for sentence  0  is  6.72408716007493e-155
bleu score for sentence  1  is  0.4657579390827988
bleu score for sentence  2  is  0.16323520519219414
bleu score for sentence  3  is  0.3958193425840313


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  4  is  1.1693071218025852e-231
bleu score for sentence  5  is  2.8918357581536547e-78
bleu score for sentence  6  is  1.2778269941762074e-231
bleu score for sentence  7  is  5.253441210115029e-155
bleu score for sentence  8  is  0.5988059642324198
bleu score for sentence  9  is  6.13274920178966e-155
bleu score for sentence  10  is  0.2757362331657834
bleu score for sentence  11  is  0.2592494576098376
bleu score for sentence  12  is  0.3265516873506877
bleu score for sentence  13  is  0.21728855279822626
bleu score for sentence  14  is  0.1851072417644606
bleu score for sentence  15  is  0.26941576910343235
bleu score for sentence  16  is  0.23409327721042972
bleu score for sentence  17  is  3.596443132911107e-78
bleu score for sentence  18  is  0.6252841934548045
bleu score for sentence  19  is  2.2891147020229563e-78
bleu score for sentence  20  is  0.7744031410142033
bleu score for sentence  21  is  0.44833867003844585
bleu score for sentence  22  is  5.411

In [ ]:
translated_sentence = translator(test_data['sentence_kaz_Cyrl'][0])[0]['translation_text']
print(translated_sentence)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


We now have 4 month old mice with diabetes but no diabetes", he added.


In [ ]:
translated_tokens = translated_sentence.split()
print(translated_tokens)

['We', 'now', 'have', '4', 'month', 'old', 'mice', 'with', 'diabetes', 'but', 'no', 'diabetes",', 'he', 'added.']


In [ ]:
bleu_score = sentence_bleu(reference_sentences[0], translated_tokens)
print(bleu_score)

6.72408716007493e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
reference_sentences[0]

[['"We',
  'now',
  'have',
  '4-month-old',
  'mice',
  'that',
  'are',
  'non-diabetic',
  'that',
  'used',
  'to',
  'be',
  'diabetic,"',
  'he',
  'added.']]

In [ ]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang="kaz_Cyrl", batch_size=64, device=device)
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in test_data['sentence_kaz_Cyrl']]
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data['sentence_eng_Latn'][i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  0  is  2.863724451169278e-78
bleu score for sentence  1  is  0.43517602947133505
bleu score for sentence  2  is  1.7073377309721624e-78
bleu score for sentence  3  is  0.3525632311542507
bleu score for sentence  4  is  0.13460697601794275
bleu score for sentence  5  is  0.3349086808413903
bleu score for sentence  6  is  3.1246366261113716e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  7  is  2.098415152738223e-155
bleu score for sentence  8  is  0.5222345260981571
bleu score for sentence  9  is  0.1460968145362594


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bleu score for sentence  10  is  0.147010155042684
bleu score for sentence  11  is  0.6413551417377991
bleu score for sentence  12  is  0.4221029565968086
bleu score for sentence  13  is  0.16852459333458833
bleu score for sentence  14  is  1.7770475530053044e-78
bleu score for sentence  15  is  0.2491070850185593
bleu score for sentence  16  is  0.11763035493591725
bleu score for sentence  17  is  0.47706165366382125
bleu score for sentence  18  is  0.45747379229381696
bleu score for sentence  19  is  0.23486091377127963
bleu score for sentence  20  is  0.2927057121559396
bleu score for sentence  21  is  0.3275571227650088
bleu score for sentence  22  is  1.2749756745208115e-78
bleu score for sentence  23  is  2.647695584065787e-155
bleu score for sentence  24  is  0.3449923615777013
bleu score for sentence  25  is  0.5461382722602006
bleu score for sentence  26  is  2.6207799533994688e-155
bleu score for sentence  27  is  0.12304670659710662
bleu score for sentence  28  is  0.1850877

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  84  is  4.556995351227316e-232
bleu score for sentence  85  is  0.42898703417794526
bleu score for sentence  86  is  0.21375047746249207
bleu score for sentence  87  is  0.23035477455994363
bleu score for sentence  88  is  0.14971022115385973
bleu score for sentence  89  is  0.23114755947377988
bleu score for sentence  90  is  0.17582103446632266
bleu score for sentence  91  is  0.32285458129919825
bleu score for sentence  92  is  0.48102547592634615
bleu score for sentence  93  is  0.18481881856839036
bleu score for sentence  94  is  1.5318926154190553e-78
bleu score for sentence  95  is  0.6298777848185599
bleu score for sentence  96  is  0.4855332614117322
bleu score for sentence  97  is  0.5491124211134414
bleu score for sentence  98  is  5.231227430537486e-155
bleu score for sentence  99  is  0.25310063863807447
bleu score for sentence  100  is  0.34800798664720095
bleu score for sentence  101  is  0.3087104594412131
bleu score for sentence  102  is  1.828

In [1]:
def read_file_and_split(filename):
    kazakh_corpus = []
    eng_corpus = []

    try:
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:  # Ensure there are exactly 3 parts: 2 sentences and a number
                    kazakh_sentence = parts[0].strip()
                    eng_sentence = parts[1].strip()
                    kazakh_corpus.append(kazakh_sentence)
                    eng_corpus.append(eng_sentence)
                else:
                    print("Invalid format in line:", line)

    except FileNotFoundError:
        print("File not found:", filename)

    return kazakh_corpus, eng_corpus

# Example usage:
kazakh_corpus, eng_corpus = read_file_and_split("kazakh_eng_corpus.txt")

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
import torch

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

# Initialize translator pipeline
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="kaz_Cyrl", tgt_lang="eng_Latn", batch_size=64, device=device)

# Load dataset
flores_dataset = load_dataset("facebook/flores", "eng_Latn-kaz_Cyrl")
test_data = kazakh_corpus

# Tokenize reference sentences
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in eng_corpus]
print(reference_sentences[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for facebook/flores contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/flores
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

[['▁over', '▁the', '▁last', '▁five', '▁years', '▁,', '▁mat', 'ernal', '▁morta', 'lity', '▁rate', '▁dec', 'reas', 'ed', '▁thre', 'ef', 'old', '▁,', '▁while', '▁the', '▁birth', 'rate', '▁is', '▁1.5', '▁times', '▁as', '▁high', '▁.']]


In [5]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data[i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

bleu score for sentence  0  is  0.3349398815984518


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  1  is  1.4031630338920504e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  2  is  3.108288608982391e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  3  is  8.510469113101058e-232
bleu score for sentence  4  is  3.679008490010746e-155
bleu score for sentence  5  is  0.20915871707586733
bleu score for sentence  6  is  3.171276006009064e-155
bleu score for sentence  7  is  0.08217174698420196
bleu score for sentence  8  is  5.981884621060668e-155
bleu score for sentence  9  is  4.061021221358935e-232
bleu score for sentence  10  is  9.929306298309508e-232
bleu score for sentence  11  is  1.012071042130996e-231
bleu score for sentence  12  is  9.788429383461836e-232
bleu score for sentence  13  is  3.743881382418666e-155
bleu score for sentence  14  is  1.0073082227171603e-231
bleu score for sentence  15  is  0
bleu score for sentence  16  is  8.614911585158347e-232
bleu score for sentence  17  is  1.163261879011684e-232
bleu score for sentence  18  is  6.484592771860512e-155
bleu score for sentence  19  is  0
bleu score for sentence  20  is  1.55422675835625e-78
bleu score for sentence  21  is  4.0342823141466

In [7]:
translator(test_data[0])[0]['translation_text']

'In the last five years , maternal mortality has decreased by about 3 times , the birth rate has increased by one and a half times .'

In [8]:
reference_sentences[0]

[['▁over',
  '▁the',
  '▁last',
  '▁five',
  '▁years',
  '▁,',
  '▁mat',
  'ernal',
  '▁morta',
  'lity',
  '▁rate',
  '▁dec',
  'reas',
  'ed',
  '▁thre',
  'ef',
  'old',
  '▁,',
  '▁while',
  '▁the',
  '▁birth',
  'rate',
  '▁is',
  '▁1.5',
  '▁times',
  '▁as',
  '▁high',
  '▁.']]

In [3]:
sentence_bleu(reference_sentences[0], reference_sentences[0][0])

1.0

In [ ]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang="eng_Latn", batch_size=64, device=device)

# Load dataset
flores_dataset = load_dataset("facebook/flores", "eng_Latn-kaz_Cyrl")
test_data = kazakh_corpus

# Tokenize reference sentences
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in eng_corpus]
print(reference_sentences[0])